In [18]:
import numpy as np
from sklearn import svm
from sklearn.linear_model import LinearRegression
x = np.array([[1,0],[0,1],[0,-1],[-1,0],[0,2],[0,-2],[-2,0]])
z = np.array([[1,-2],[4,-5],[4,-1],[5,-2],[7,-7],[7,1],[7,1]])
y = np.array([-1,-1,-1,1,1,1,1])
clf = svm.SVC(kernel = 'poly', degree = 2, gamma = 1, coef0 = 1)
clf.fit(x,y)


SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=1,
    decision_function_shape='ovr', degree=2, gamma=1, kernel='poly',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [33]:
clf.support_vectors_
print(np.sum(abs(clf.dual_coef_)))
print('intercept: ',clf._intercept_)
print('alpha*y :',clf.dual_coef_)

2.8142453490044304
intercept:  [1.66633088]
alpha*y : [[-0.59647182 -0.81065085  0.8887034   0.20566488  0.31275439]]


In [2]:
import numpy as np
from cvxopt import matrix, solvers 

def kernel(x1,x2):
    k = (1 + np.dot(x1.T,x2))**2
    return k

def dualsvm(x,y):
    N = len(x)
    Q = np.zeros((N,N))
    for i in range(N):
        for j in range(N):
            Q[i][j] = y[i]*y[j]*kernel(x[i],x[j])
    Q = matrix(Q, tc = 'd')
    p = matrix(np.array([-1]*N), tc = 'd')
    c = matrix(np.array([0]*(N+2)), tc = 'd')
    A = np.zeros((N+2,N))
    for j in range(N):
        A[0][j] = -y[j]
        A[1][j] = y[j]
        A[j+2][j] = -1
    A = matrix(A, tc = 'd')
    sol = solvers.qp(Q, p, A, c)
    alpha = []
    for i in range(N):
        alpha.append(sol['x'][i])
    alpha = np.asarray(alpha)
    return alpha

x = np.array([[1,0],
              [0,1],
              [0,-1],
              [-1,0],
              [0,2],
              [0,-2],
              [-2,0]])
y = np.array([-1,-1,-1,1,1,1,1])
alpha = dualsvm(x,y)
print(alpha)

b = y[1]
for i in range(N):
    b -= y[i]*alpha[i]*kernel(x[i], y[i])

"import numpy as np\nfrom cvxopt import matrix, solvers \n\ndef kernel(x1,x2):\n    k = (1 + np.dot(x1.T,x2))**2\n    return k\n\ndef dualsvm(x,y):\n    N = len(x)\n    Q = np.zeros((N,N))\n    for i in range(N):\n        for j in range(N):\n            Q[i][j] = y[i]*y[j]*kernel(x[i],x[j])\n    Q = matrix(Q, tc = 'd')\n    p = matrix(np.array([-1]*N), tc = 'd')\n    c = matrix(np.array([0]*(N+2)), tc = 'd')\n    A = np.zeros((N+2,N))\n    for j in range(N):\n        A[0][j] = -y[j]\n        A[1][j] = y[j]\n        A[j+2][j] = -1\n    A = matrix(A, tc = 'd')\n    sol = solvers.qp(Q, p, A, c)\n    alpha = []\n    for i in range(N):\n        alpha.append(sol['x'][i])\n    alpha = np.asarray(alpha)\n    return alpha\n\nx = np.array([[1,0],\n              [0,1],\n              [0,-1],\n              [-1,0],\n              [0,2],\n              [0,-2],\n              [-2,0]])\ny = np.array([-1,-1,-1,1,1,1,1])\nalpha = dualsvm(x,y)\nprint(alpha)\n\nb = y[1]\nfor i in range(N):\n    b -= y[i]

In [32]:
import numpy as np
from sklearn.svm import SVC

def getdata():
    train = np.genfromtxt('train1.txt')
    test = np.genfromtxt('test1.txt')
    m,n = train.shape
    m1,n1 = test.shape
    x = np.c_[np.ones((m,1)), train[:,1:n]]
    y = train[:,0]
    testx = np.c_[np.ones((m1,1)), test[:,1:n]]
    testy = test[:,0]
    
    return x, y, testx, testy

def errorRate(yhat, y):
    N = len(y)
    errorSum = np.sum(yhat != y)
    return errorSum / N

def preprocess(y):
    y[np.where(y != 0)] = -1
    y[np.where(y == 0)] = 1
    return y

def p15():
    x, y, testx, testy = getdata()
    ytemp = y.copy()
    ytemp = preprocess(ytemp)
    clf = SVC(C = 0.01, kernel = 'linear').fit(x, ytemp)
    w_norm = np.linalg.norm(clf.coef_)
    return w_norm

def p1617():
    x, y, testx, testy = getdata()
    lis = [0, 2, 4, 6, 8]
    for i in lis:
        ytemp = y.copy()
        ytemp[np.where(ytemp != i)] = -1
        ytemp[np.where(ytemp == i)] = 1
        clf = SVC(C = 0.01, kernel = 'poly', degree = 2, gamma = 1, coef0 = 1).fit(x, ytemp)
        ypred = clf.predict(x)
        e_rate = errorRate(ypred, ytemp)
        alpha_sum = np.sum(abs(clf.dual_coef_))
        print('{0} versus non {0}, Ein = {1}, sum of alpha = {2}'.format(i, e_rate, alpha_sum))

def p20():
    

In [57]:
x, y, testx, testy = getdata()
ytemp = y.copy()
g_list = [1, 10, 1e2, 1e3, 1e4]
ytemp = preprocess(ytemp)
row, col = x.shape
Eval = np.zeros((len(g_list),))
for i in range(5):
    for j in range(100):
        pos = np.random.permutation(row)
        val_x = x[pos[:1000],:]
        val_y = ytemp[pos[:1000]]
        train_x = x[pos[:1000],:]
        train_y = ytemp[pos[:1000]]
        clf = SVC(C = 0.1, kernel = 'rbf', gamma = g_list[i]).fit(train_x, train_y)
        yhat = clf.predict(val_x)
        error = errorRate(yhat, val_y)
        Eval[i] += error

        

array([16.274, 12.679, 16.422, 16.573, 16.41 ])

In [58]:
Eval / 100

array([0.16274, 0.12679, 0.16422, 0.16573, 0.1641 ])

In [52]:
g_list = [1, 10, 1e2, 1e3, 1e4]
g_list[0]

1